In [2]:
!pip install cohere umap-learn altair datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 7.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 6.7 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import cohere
import pandas as pd

In [4]:
#cohere_api_key = <cohere_api_key>"
co = cohere.Client(cohere_api_key)

In [5]:
three_words = pd.DataFrame({'text':
  [
      'joy',
      'happiness',
      'potato'
  ]})

three_words

,text
0,joy
1,happiness
2,potato


In [6]:
three_words_emb = co.embed(texts=list(three_words['text']),
                           model='embed-english-v2.0').embeddings

In [7]:
word_1 = three_words_emb[0]
word_2 = three_words_emb[1]
word_3 = three_words_emb[2]

In [8]:
len(word_1)

4096

## Sentence Embeddings

In [9]:
sentences = pd.DataFrame({'text':
  [
   'Where is the world cup?',
   'The world cup is in Qatar',
   'What color is the sky?',
   'The sky is blue',
   'Where does the bear live?',
   'The bear lives in the the woods',
   'What is an apple?',
   'An apple is a fruit',
  ]})

sentences

,text
0,Where is the world cup?
1,The world cup is in Qatar
2,What color is the sky?
3,The sky is blue
4,Where does the bear live?
5,The bear lives in the the woods
6,What is an apple?
7,An apple is a fruit


In [10]:
emb = co.embed(texts=list(sentences['text']),
               model='embed-english-v2.0').embeddings

# Explore the 10 first entries of the embeddings of the 3 sentences:
for e in emb:
    print(e[:3])

[0.27319336, -0.37768555, -1.0273438]
[0.49804688, 1.2236328, 0.4074707]
[-0.23571777, -0.9375, 0.9614258]
[0.08300781, -0.32080078, 0.9272461]
[0.49780273, -0.35058594, -1.6171875]
[1.2294922, -1.3779297, -1.8378906]
[0.15686035, -0.92041016, 1.5996094]
[1.0761719, -0.7211914, 0.9296875]


In [11]:
len(emb[0])

4096

In [41]:
import umap
import altair as alt
import pandas as pd

def umap_plot(sentences, emb, labels=None):
    """
    Create a UMAP plot to visualize text embeddings.

    Parameters:
    - sentences: List of text sentences or documents.
    - emb: List of corresponding embeddings (e.g., word embeddings or document embeddings).
    - labels: Optional list of labels for coloring points in the plot.

    Returns:
    - Altair chart object for an interactive UMAP plot.
    """
    # Perform UMAP dimensionality reduction
    reducer = umap.UMAP(n_neighbors=2, min_dist=0.0, metric='cosine')
    #reducer = umap.UMAP()

    embedding = reducer.fit_transform(emb)

    # Create a DataFrame for visualization
    df = pd.DataFrame({'x': embedding[:, 0], 'y': embedding[:, 1]})
    if labels:
        df['label'] = labels

    # Create UMAP scatter plot using Altair
    scatter = alt.Chart(df).mark_circle(size=60, opacity=0.6).encode(
        x='x',
        y='y',
        #tooltip=labels if labels else [],
        #tooltip=['sentences'] + (labels if labels else []),
    )

    if labels:
        scatter = scatter.encode(
            color=alt.Color('label:N', scale=alt.Scale(scheme='category20')),
        )

    return scatter

# Example usage:
# Replace 'sentences' and 'emb' with your actual data
# umap_plot(sentences, emb, labels=your_labels).interactive()


In [42]:
chart = umap_plot(sentences, emb)

In [43]:
chart.interactive()

alt.Chart(...)